On cherche à voir quels filtres fonctionnent le mieux sur notre mini corpus de phrases alignées correctement

In [13]:
with open("./data/aligned/en.txt") as f_en, open("./data/aligned/fr.txt") as f_fr:
    en_sentences = [l.strip() for l in f_en]
    fr_sentences = [l.strip() for l in f_fr]
original_pairs = list(zip(en_sentences, fr_sentences))

print(original_pairs)
print(len(original_pairs))

[('Hi, my name is Pierre and i love eating apples.', "Bonjour, je m'appelle Pierre et j'adore manger des pommes."), ('Last night, i took the bus 396 to go home from the cinema, it was dark.', "Hier soir, j'ai pris le bus 396 pour rentrer du cinéma, il faisait nuit noire."), ('This artist is my favorite artist, i discovered him during a concert in Las Vegas. His album "Vanilla" is a masterpiece.', 'Cet artiste est mon artiste préféré, je l\'ai découvert lors d\'un concert à Las Vegas. Son album "Vanilla" est un chef d\'oeuvre.'), ("Here is my email address: jean.pierre@gmail.com Please don't spam me, I hate it.", "Voici mon adresse mail : jean.pierre@gmail,com ne me spamer pas s'il vous plait je déteste ça."), ('The PS3 was the best-selling console during the period 2005-2013, with the Xbox 360 slightly behind.', 'La ps3 est la console la plus vendue pendant la période 2005-2013, la xbox 360 quant à elle est légèrement derrière.'), ('Whoa!!! Where are you going like that ? I almost fell

On va générer plusieurs fichiers yaml de config pour la librairie opusfilter, avec des treshold différents

In [ ]:
## cas pour générer des fichiers yaml avec treshold différents (ici pour le lenghtRatio)
import yaml

thresholds = [0.5, 1.0, 1.5, 2.0]

for threshold in thresholds:
    with open('./data/settings_yaml/config.yaml', 'r') as f:
        config = yaml.safe_load(f)

   
    filters = config['steps'][0]['parameters']['filters']
    for f in filters:
        if 'LengthRatioFilter' in f:
            f['LengthRatioFilter']['threshold'] = threshold

    # Mettre à jour le dossier de sortie pour éviter les conflits
    config['common']['output_directory'] = f"data/filtered_{threshold}"


    config_path = f"data/settings_yaml/config_{threshold}.yaml"
    with open(config_path, 'w') as f:
        yaml.safe_dump(config, f)



In [15]:
import subprocess

thresholds = [0.5, 1.0, 1.5, 2.0]

for threshold in thresholds:
    config_path = f"data/settings_yaml/config_{threshold}.yaml"
    print(f"Running opusfilter with threshold {threshold}...")
    subprocess.run(["opusfilter", "--overwrite", config_path])


Running opusfilter with threshold 0.5...


INFO:opusfilter.opusfilter:Running step 1: filter
50it [00:00, 66980.26it/s]


Running opusfilter with threshold 1.0...


INFO:opusfilter.opusfilter:Running step 1: filter
50it [00:00, 66093.67it/s]


Running opusfilter with threshold 1.5...


INFO:opusfilter.opusfilter:Running step 1: filter
50it [00:00, 16418.63it/s]


Running opusfilter with threshold 2.0...


INFO:opusfilter.opusfilter:Running step 1: filter
50it [00:00, 19035.60it/s]


In [ ]:
# Résultats par seuil
results = {}
import gzip
for threshold in thresholds:
    path = f"./data/filtered_{threshold}"
    with gzip.open(f"{path}/en.filtered.gz", "rt", encoding="utf-8") as f_en_filt, \
         gzip.open(f"{path}/fr.filtered.gz", "rt", encoding="utf-8") as f_fr_filt:
        filtered_pairs = set(zip(
            [l.strip() for l in f_en_filt],
            [l.strip() for l in f_fr_filt]
        ))

    removed_pairs = [pair for pair in original_pairs if pair not in filtered_pairs]
    results[threshold] = len(removed_pairs)

    print(f"\n=== Threshold {threshold} ===")
    print(f"{len(removed_pairs)} paires supprimées.")
    print("Exemples :")
    for pair in removed_pairs[:5]:  
        print(f"- EN: {pair[0]}")
        print(f"  FR: {pair[1]}")



=== Threshold 0.5 ===
50 paires supprimées.
Exemples :
- EN: Hi, my name is Pierre and i love eating apples.
  FR: Bonjour, je m'appelle Pierre et j'adore manger des pommes.
- EN: Last night, i took the bus 396 to go home from the cinema, it was dark.
  FR: Hier soir, j'ai pris le bus 396 pour rentrer du cinéma, il faisait nuit noire.
- EN: This artist is my favorite artist, i discovered him during a concert in Las Vegas. His album "Vanilla" is a masterpiece.
  FR: Cet artiste est mon artiste préféré, je l'ai découvert lors d'un concert à Las Vegas. Son album "Vanilla" est un chef d'oeuvre.
- EN: Here is my email address: jean.pierre@gmail.com Please don't spam me, I hate it.
  FR: Voici mon adresse mail : jean.pierre@gmail,com ne me spamer pas s'il vous plait je déteste ça.
- EN: The PS3 was the best-selling console during the period 2005-2013, with the Xbox 360 slightly behind.
  FR: La ps3 est la console la plus vendue pendant la période 2005-2013, la xbox 360 quant à elle est légè